# Parcellate XCP-D denoised BOLD with a custom atlas

Users may want to parcellate their denoised BOLD data with an atlas that isn't built into XCP-D.
They _could_ simply use a NiftiLabelsMasker to parcellate the data, but this would ignore some of the extra steps that XCP-D applies, including the removal of missing voxels/vertices and the application of a minimum coverage threshold.

This notebook has steps to warp the atlas to the same space and resolution as the denoised BOLD data.

In [ ]:
import os

from xcp_d.interfaces.ants import ApplyTransforms
from xcp_d.interfaces.connectivity import CiftiConnect, NiftiConnect
from xcp_d.interfaces.nilearn import IndexImage
from xcp_d.interfaces.workbench import CiftiCreateDenseFromTemplate, CiftiParcellate
from xcp_d.utils.utils import get_std2bold_xfms

In [ ]:
xcpd_dir = "/Users/taylor/Documents/datasets/ds003643/derivatives/xcp_d/sub-EN100/func"
fmriprep_dir = "/Users/taylor/Documents/datasets/ds003643/derivatives/fmriprep/sub-EN100/func"

mask = os.path.join(fmriprep_dir, "sub-EN100_task-lppEN_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz")
temporal_mask = os.path.join(xcpd_dir, "sub-EN100_task-lppEN_run-1_outliers.tsv")

min_coverage = 0.5  # use the same threshold you used for the XCP-D call
correlate = True

In [ ]:
nifti_file = os.path.join(xcpd_dir, "sub-EN100_task-lppEN_run-1_space-MNI152NLin6Asym_desc-denoised_bold.nii.gz")
nifti_atlas = "tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-02_desc-100Parcels17Networks_dseg.nii.gz"

# Step 1. Mock up an atlas labels file
# The file must have two columns: index and label
# I decided to just use one of the ones from XCP-D.
nifti_atlas_labels = "atlas-Schaefer2018v0143_desc-100Parcels17Networks_dseg.tsv"

# Step 2. Warp the atlas to the same space as the BOLD file
transform_files = get_std2bold_xfms(nifti_file)

grab_first_volume = IndexImage(
    in_file=nifti_file,
    index=0,
)
gfv_results = grab_first_volume.run()

warp_atlases_to_bold_space = ApplyTransforms(
    interpolation="GenericLabel",
    input_image_type=3,
    dimension=3,
    reference_image=gfv_results.outputs.out_file,
    input_image=nifti_atlas,
    transforms=transform_files,
)
watbs_results = warp_atlases_to_bold_space.run()
warped_nifti_atlas = watbs_results.outputs.output_image

# Step 3. Parcellate the BOLD file
interface = NiftiConnect(
    filtered_file=nifti_file,
    mask=mask,
    temporal_mask=temporal_mask,
    atlas=warped_nifti_atlas,
    atlas_labels=nifti_atlas_labels,
    min_coverage=min_coverage,
    correlate=True,
)
results = interface.run()

# These are your outputs
timeseries_file = results.outputs.timeseries
correlations_file = results.outputs.correlations

In [ ]:
cifti_file = "sub-01_task-imagery_run-01_space-fsLR_den-91k_desc-denoised_bold.dtseries.nii"
cifti_atlas = "tpl-fsLR_atlas-Schaefer2018v0143_den-32k_desc-100Parcels17Networks_dseg.dlabel.nii"

# Step 1. Mock up an atlas labels file
# The file must have three columns: index, label, and cifti_label
# The cifti_label column values *must* match the internal labels in the CIFTI file
# I decided to just use one of the ones from XCP-D.
cifti_atlas_labels = "atlas-Schaefer2018v0143_desc-100Parcels17Networks_dseg.tsv"

# Step 2. Parcellate the CIFTI atlas
resample_atlas_to_data = CiftiCreateDenseFromTemplate(
    template_cifti=cifti_file,
    label=cifti_atlas,
)
ratd_results = resample_atlas_to_data.run()

parcellate_atlas = CiftiParcellate(
    direction="COLUMN",
    only_numeric=True,
    atlas_label=cifti_atlas,
    in_file=ratd_results.outputs.cifti_out,
    out_file="parcellated_atlas.pscalar.nii",
)
pa_results = parcellate_atlas.run()
parcellated_cifti_atlas = pa_results.outputs.out_file

# Step 3. Parcellate the BOLD file
interface = CiftiConnect(
    data_file=cifti_file,
    temporal_mask=temporal_mask,
    atlas_file=cifti_atlas,
    parcellated_atlas=parcellated_cifti_atlas,
    atlas_labels=cifti_atlas_labels,
    min_coverage=min_coverage,
    correlate=True,
)
results = interface.run()

# These are your outputs
timeseries_file = results.outputs.timeseries
timeseries_cifti_file = results.outputs.timeseries_ciftis
correlations_file = results.outputs.correlations
correlations_cifti_file = results.outputs.correlation_ciftis